<h1 style="color:orange">This Notebook was done in colaborate between Shibab Ahsan(79826) and Sebastian Rix(71411)</h1>

## Exercise 1

For this exercise, we will use the `adult` dataset (available on moodle or from the [UCI Machine Learning repository](https://archive.ics.uci.edu/dataset/2/adult)). Do the following:

1. Load in the dataset and correct the error in the income column (replace the "." with the empty string such that there are only two categories).
2. Create an X dataset using the variables "age", "workclass", "education", "occupation", "race", "sex", "hours-per-week". For the categorical variables with missing values, replace the missing values with a new category "Unknown". Also replace any values that are "?" with the value "Unknown (using `str.replace`, for instance)
3. Turn the five categorical variables in X into dummy variables and remove the original five variables (This will probably give you around 44 columns in X)
4. Create the response variable y, such that it is 1 if the `income` variable in the adult dataset is `>50K` and 0 if the value is `<=50K`.
5. Do a train-test split with 30% of the data for test (using `random_state=123`) and train a `XGBoost` classification model on the training data.
6. Evaluate your models using various evaluation metrics and look at the confusion matrix of your model.
7. To be able to calculate the various fairness metrics in regard to the variable `sex`, we need to construct two separate confusion matrices for the test dataset, one for `female` and one for `male`. First, create separate test sets for `female` and `male` as well as the predicted values for each gender. That is, create `X_test_female`, `X_test_male`, `y_test_female`, `y_test_male`, `y_pred_female`, and `y_pred_male`. (Hint: You can create `X_test_female` by `X_test_female = X_test[X_test["sex_Male"] == 0]` and `y_test_male` by `y_test_male = y_test[X_test["sex_Male"] == 1]`, for instance.)
8. Calculate the accuracy for female and male for the XGBoost model and comment on the results.
9. We can now create the True Positive (TP), True Negative (TN), False Positive (FP), and False Negative (FN) for each gender. That is, calculate the eight values `TP_f`, `TN_f`, `FP_f`, `FN_f`, `TP_m`, `TN_m`, `FP_m`, and `FN_m`. (Hint: You can calculate the False Positive for female (FP_f) by `FP_f = sum((y_test_female == 0) & (y_pred_female == 1))`.)
10. Is there error rate balance across different genders, i.e. are the false positive rate (FPR) and false negative rate (FNR) the same across the two genders?
11. Is there predictive parity across different genders?
12. Is there Statistical parity across different genders?
13. [Discussion question] Can your any of your models be used to make fair salary predictions?
14. [Discussion question] In what sense is the `adult` dataset biased (unfair)?
15. [Discussion question] If the dataset is biased, where could the bias potentially come from?
16. [Optional] If you balance the number of males and females in the dataset (like we balanced the response variable in the Churn example), will you model become more fair?

### Task 1: Load in the dataset and correct the error in the income column (replace the "." with the empty string such that there are only two categories).


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  
import seaborn as sns



In [2]:
df = pd.read_csv("adult.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
df['income'] = df['income'].str.strip().replace({'<=50K.': '<=50K', '>50K.': '>50K'})
print(df['income'].unique())




['<=50K' '>50K']


#### Task 2: Create an X dataset using the variables "age", "workclass", "education", "occupation", "race", "sex", "hours-per-week". For the categorical variables with missing values, replace the missing values with a new category "Unknown". Also replace any values that are "?" with the value "Unknown (using `str.replace`, for instance)


Step 1: Select the variables mentioned to build X<br>
Step 2: Replace any '?' values in the categorical columns with 'Unknown'<br>
Step 3: Fill any remaining missing values in those categorical columns with 'Unknown'

In [4]:
# Step 1: create X dataset with only the selected columns
X = df[["age", "workclass", "education", "occupation", "race", "sex", "hours-per-week"]].copy()

# Step 2: replace any "?" values in the categorical columns with "Unknown"
categorical_cols = ["workclass", "education", "occupation", "race", "sex"]
for col in categorical_cols:
    X[col] = X[col].str.replace("?", "Unknown")

# Step 3: fill missing values (NaN) in the categorical columns with "Unknown"
X[categorical_cols] = X[categorical_cols].fillna("Unknown")
print(X)

print(X.columns.tolist())
print(len(X.columns))


       age         workclass  education         occupation  \
0       39         State-gov  Bachelors       Adm-clerical   
1       50  Self-emp-not-inc  Bachelors    Exec-managerial   
2       38           Private    HS-grad  Handlers-cleaners   
3       53           Private       11th  Handlers-cleaners   
4       28           Private  Bachelors     Prof-specialty   
...    ...               ...        ...                ...   
48837   39           Private  Bachelors     Prof-specialty   
48838   64           Unknown    HS-grad            Unknown   
48839   38           Private  Bachelors     Prof-specialty   
48840   44           Private  Bachelors       Adm-clerical   
48841   35      Self-emp-inc  Bachelors    Exec-managerial   

                     race     sex  hours-per-week  
0                   White    Male              40  
1                   White    Male              13  
2                   White    Male              40  
3                   Black    Male              

### Task3: Turn the five categorical variables in X into dummy variables and remove the original five variables (This will probably give you around 44 columns in X)


In [5]:
# Re-create X from df (Task 2 with cleaning)
X = df[["age", "workclass", "education", "occupation", "race", "sex", "hours-per-week"]]

X[["workclass", "education", "occupation", "race", "sex"]] = \
X[["workclass", "education", "occupation", "race", "sex"]].replace("?", "Unknown").fillna("Unknown")


# List of categorical variables to convert
categorical_cols = ["workclass", "education", "occupation", "race", "sex"]

# Step 1: create dummy variables for the categorical columns
dummies = pd.get_dummies(X[categorical_cols])

# Step 2: drop the original categorical columns from X
X = X.drop(columns=categorical_cols)

# Step 3: concatenate the dummy columns back into X
X = pd.concat([X, dummies], axis=1)

print(X)

       age  hours-per-week  workclass_Federal-gov  workclass_Local-gov  \
0       39              40                  False                False   
1       50              13                  False                False   
2       38              40                  False                False   
3       53              40                  False                False   
4       28              40                  False                False   
...    ...             ...                    ...                  ...   
48837   39              36                  False                False   
48838   64              40                  False                False   
48839   38              50                  False                False   
48840   44              40                  False                False   
48841   35              60                  False                False   

       workclass_Never-worked  workclass_Private  workclass_Self-emp-inc  \
0                       False      

C:\Users\sebas\AppData\Local\Temp\ipykernel_7328\4003560175.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[["workclass", "education", "occupation", "race", "sex"]] = \


In [6]:
print(X.columns.tolist())

['age', 'hours-per-week', 'workclass_Federal-gov', 'workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc', 'workclass_State-gov', 'workclass_Unknown', 'workclass_Without-pay', 'education_10th', 'education_11th', 'education_12th', 'education_1st-4th', 'education_5th-6th', 'education_7th-8th', 'education_9th', 'education_Assoc-acdm', 'education_Assoc-voc', 'education_Bachelors', 'education_Doctorate', 'education_HS-grad', 'education_Masters', 'education_Preschool', 'education_Prof-school', 'education_Some-college', 'occupation_Adm-clerical', 'occupation_Armed-Forces', 'occupation_Craft-repair', 'occupation_Exec-managerial', 'occupation_Farming-fishing', 'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct', 'occupation_Other-service', 'occupation_Priv-house-serv', 'occupation_Prof-specialty', 'occupation_Protective-serv', 'occupation_Sales', 'occupation_Tech-support', 'occupation_Transport-moving', 'occupation

### Task 4: Create the response variable y, such that it is 1 if the `income` variable in the adult dataset is `>50K` and 0 if the value is `<=50K`.


In [7]:

# Show the original income column and the new y variable together (first 10 rows)


preview = pd.DataFrame({"income_original": df["income"],"y_converted": (df["income"] == ">50K").astype(int)})

y = (df["income"] == ">50K").astype(int) # Target variable

print(preview.head(10))

  income_original  y_converted
0           <=50K            0
1           <=50K            0
2           <=50K            0
3           <=50K            0
4           <=50K            0
5           <=50K            0
6           <=50K            0
7            >50K            1
8            >50K            1
9            >50K            1


### Task 5: Do a train-test split with 30% of the data for test (using `random_state=123`) and train a `XGBoost` classification model on the training data.


In [8]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# Train-test split (30% test, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=123
)

# Create and train the XGBoost classifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Evaluate basic accuracy on the test set
accuracy = model.score(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.8228


c:\Users\sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [17:12:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


### Task 6: Evaluate your models using various evaluation metrics and look at the confusion matrix of your model.


In [9]:
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

# Predictions on the test set
y_pred = model.predict(X_test)

# Evaluation metrics
accuracy  = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
conf_mat  = confusion_matrix(y_test, y_pred)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print("Confusion Matrix:")
print(conf_mat)


Accuracy:  0.8228
Precision: 0.6825
Confusion Matrix:
[[10392   774]
 [ 1823  1664]]


##### Task 7: To be able to calculate the various fairness metrics in regard to the variable `sex`, we need to construct two separate confusion matrices for the test dataset, one for `female` and one for `male`. First, create separate test sets for `female` and `male` as well as the predicted values for each gender. That is, create `X_test_female`, `X_test_male`, `y_test_female`, `y_test_male`, `y_pred_female`, and `y_pred_male`. (Hint: You can create `X_test_female` by `X_test_female = X_test[X_test["sex_Male"] == 0]` and `y_test_male` by `y_test_male = y_test[X_test["sex_Male"] == 1]`, for instance.)


Step 1: create gender-specific test sets and predictions

In [10]:


# Female subset (sex_Male == 0)
X_test_female = X_test[X_test["sex_Male"] == 0]
y_test_female = y_test[X_test["sex_Male"] == 0]
y_pred_female = model.predict(X_test_female)

# Male subset (sex_Male == 1)
X_test_male = X_test[X_test["sex_Male"] == 1]
y_test_male = y_test[X_test["sex_Male"] == 1]
y_pred_male = model.predict(X_test_male)

# Quick sanity check (optional)
print("Female test size:", len(X_test_female))
print("Male test size:  ", len(X_test_male))


Female test size: 4854
Male test size:   9799


Step 2: create confusion matrices 

In [11]:

from sklearn.metrics import confusion_matrix

# Confusion matrix for females
conf_mat_female = confusion_matrix(y_test_female, y_pred_female)

# Confusion matrix for males
conf_mat_male = confusion_matrix(y_test_male, y_pred_male)

print("Female Confusion Matrix:")
print(conf_mat_female)

print("\nMale Confusion Matrix:")
print(conf_mat_male)


Female Confusion Matrix:
[[4311   59]
 [ 427   57]]

Male Confusion Matrix:
[[6081  715]
 [1396 1607]]


### Task 8: Calculate the accuracy for female and male for the XGBoost model and comment on the results.


In [12]:
from sklearn.metrics import accuracy_score

# Accuracy for females
accuracy_female = accuracy_score(y_test_female, y_pred_female)

# Accuracy for males
accuracy_male = accuracy_score(y_test_male, y_pred_male)

print(f"Female Accuracy: {accuracy_female:.4f}")
print(f"Male Accuracy:   {accuracy_male:.4f}")


Female Accuracy: 0.8999
Male Accuracy:   0.7846


The model clearly performs better for females (~0.90 accuracy) than for males (~0.78).<br>
That difference is pretty big and could be a sign of unfairness, so we should dig deeper (e.g. look at precision/recall or confusion matrices) to understand what’s driving it.

##### Task 9: We can now create the True Positive (TP), True Negative (TN), False Positive (FP), and False Negative (FN) for each gender. That is, calculate the eight values `TP_f`, `TN_f`, `FP_f`, `FN_f`, `TP_m`, `TN_m`, `FP_m`, and `FN_m`. (Hint: You can calculate the False Positive for female (FP_f) by `FP_f = sum((y_test_female == 0) & (y_pred_female == 1))`.)


In [13]:
#Female group
TP_f = sum((y_test_female == 1) & (y_pred_female == 1))
TN_f = sum((y_test_female == 0) & (y_pred_female == 0))
FP_f = sum((y_test_female == 0) & (y_pred_female == 1))
FN_f = sum((y_test_female == 1) & (y_pred_female == 0))

#Male group
TP_m = sum((y_test_male == 1) & (y_pred_male == 1))
TN_m = sum((y_test_male == 0) & (y_pred_male == 0))
FP_m = sum((y_test_male == 0) & (y_pred_male == 1))
FN_m = sum((y_test_male == 1) & (y_pred_male == 0))

print("Female — TP:", TP_f, "TN:", TN_f, "FP:", FP_f, "FN:", FN_f)
print("Male   — TP:", TP_m, "TN:", TN_m, "FP:", FP_m, "FN:", FN_m)


Female — TP: 57 TN: 4311 FP: 59 FN: 427
Male   — TP: 1607 TN: 6081 FP: 715 FN: 1396


#### Task 10: Is there error rate balance across different genders, i.e. are the false positive rate (FPR) and false negative rate (FNR) the same across the two genders?


In [14]:
# False Positive Rate (FPR) 
FPR_f = FP_f / (FP_f + TN_f)
FPR_m = FP_m / (FP_m + TN_m)

# False Negative Rate (FNR)
FNR_f = FN_f / (FN_f + TP_f)
FNR_m = FN_m / (FN_m + TP_m)

print(f"Female FPR: {FPR_f:.4f}, Female FNR: {FNR_f:.4f}")
print(f"Male   FPR: {FPR_m:.4f}, Male   FNR: {FNR_m:.4f}")


Female FPR: 0.0135, Female FNR: 0.8822
Male   FPR: 0.1052, Male   FNR: 0.4649


No, the model isn’t balanced across genders.<br>
For females, it almost never predicts “>50K” when it shouldn’t (low FPR), but it misses a lot of women who actually do earn >50K (very high FNR).<br>
For males, it makes more false positives, but also catches more of the true positives (lower FNR).<br>
So the two error rates are very different → meaning the model behaves differently for men and women.<br>

#### Task 11: Is there predictive parity across different genders?


In [15]:
from sklearn.metrics import precision_score

# Precision for females
precision_female = precision_score(y_test_female, y_pred_female)

# Precision for males
precision_male = precision_score(y_test_male, y_pred_male)

print(f"Female Precision: {precision_female:.4f}")
print(f"Male Precision:   {precision_male:.4f}")


Female Precision: 0.4914
Male Precision:   0.6921


No, predictive parity is not satisfied. The precision for males (0.69) is clearly higher than the precision for females (0.49), which means the model’s positive predictions are more reliable for men than for women.

#### Task 12: Is there Statistical parity across different genders?


In [16]:
# Statistical Parity (probability of positive prediction)

prob_female = (y_pred_female == 1).mean()
prob_male   = (y_pred_male == 1).mean()

print(f"Probability of positive prediction (Female): {prob_female:.4f}")
print(f"Probability of positive prediction (Male):   {prob_male:.4f}")


Probability of positive prediction (Female): 0.0239
Probability of positive prediction (Male):   0.2370


statistical parity does not hold.<br>
The model is much more likely to assign a positive prediction to male individuals compared to female individuals.

#### Task13: [Discussion question] Can your any of your models be used to make fair salary predictions?


No the model isn’t fair enough to be used for salary predictions.<br>
It treats males and females very differently (especially in terms of false negatives and how often it predicts high income), so using it in the real world would likely reinforce existing bias.<br>
We’d need to apply fairness adjustments or redesign the model before using it.<br>

#### Task 14: [Discussion question] In what sense is the `adult` dataset biased (unfair)?

The Adult dataset reflects real-world historical data, and those data already contain social and economic inequalities.<br>
For example, women and certain groups have lower incomes more often, even if they have similar qualifications.<br>
Because the dataset captures these past inequalities, any model trained on it learns and reproduces that unfairness — which makes the dataset itself biased.<br>

#### Task 15: [Discussion question] If the dataset is biased, where could the bias potentially come from?

The bias could come from the way the data was collected and from historical inequalities in society.<br>
For example, women or minority groups may have had less access to high-paying jobs due to discrimination, education gaps, or hiring practices.<br>
So even if the data is “real,” it still reflects an unfair reality — and that bias gets passed into the dataset.<br>